---
title: "GIOZA Case Study: Production Analysis"
format:
  html:
    toc: true
    toc-title: Contents
    toc-depth: 4
    code-fold: show
    self-contained: true
jupyter: python3
---

# Loading Packages and Data

In [1]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from plotnine import *

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
factory_prod = pd.read_csv('data/production.csv')
# convert predictors into proper datatype
factory_prod['Confirmation'] = factory_prod['Confirmation'].astype(str)
factory_prod['Ordre'] = factory_prod['Ordre'].astype(str)
factory_prod['Poste de travail'] = factory_prod['Poste de travail'].astype(str)
factory_prod['Centre de coûts'] = factory_prod['Centre de coûts'].astype(str)
factory_prod['sku'] = factory_prod['sku'].astype(str)

factory_prod['at'] = pd.to_datetime(factory_prod['at'])
factory_prod = factory_prod.sort_values(by=['at'], ascending=True).reset_index(drop=True)

print( factory_prod.dtypes )

Confirmation                     object
Ordre                            object
Poste de travail                 object
Centre de coûts                  object
sku                              object
units                             int64
weight_kg                       float64
scrap                             int64
group                            object
at                  datetime64[ns, UTC]
dtype: object


# Units

In [3]:
data_unit = factory_prod.drop(['Confirmation', 'Ordre', 'at'], axis=1)
data_unit_num = data_unit.select_dtypes(include=['int64', 'float64']).drop(['units'], axis=1)
data_unit_cat = data_unit.select_dtypes(exclude=['int64', 'float64'])
n_train = math.floor(0.9 * len(data_unit))

# split into train / test
trainX_num = data_unit_num[0:n_train]
trainY = data_unit[['units']][0:n_train]

testX_num = data_unit_num[n_train:]
testY = data_unit[['units']][n_train:]

# apply normalization and one-hot encoding
scaler = MinMaxScaler()
trainX_num = scaler.fit_transform(trainX_num)
testX_num = scaler.transform(testX_num)

data_unit_cat = pd.get_dummies(data_unit_cat)

# get final train and test set
data_scaled = pd.DataFrame(
    np.concatenate((np.concatenate((trainX_num, testX_num), axis=0), data_unit_cat), axis=1),
    columns=list(data_unit_num.columns) + list(data_unit_cat.columns)
)

trainX = data_scaled[0:n_train]
testX = data_scaled[n_train:]

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(8443, 534) (8443, 1)
(939, 534) (939, 1)


In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [5]:
# fit decision tree model
model_tree = DecisionTreeRegressor(max_depth=10)
model_tree.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_tree = model_tree.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_tree = model_tree.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 16423319.083621074
Testing Error: 15995566.473672068


In [6]:
model_tree_coefs = pd.DataFrame(
    model_tree.feature_importances_,
    index=list(trainX.columns), columns=['coef']
)
model_tree_coefs.sort_values(by=['coef'], key=abs, ascending=False)

,coef
weight_kg,4.697500e-01
sku_1663717,1.167535e-01
group_4G130160,9.400354e-02
sku_762385,5.804926e-02
sku_762388,4.459438e-02
sku_1779728,4.249520e-02
sku_1099287,3.534159e-02
sku_1099285,2.888065e-02
sku_1689096,2.853637e-02
sku_1390940,2.711134e-02


In [7]:
# fit random forest model
model_forest = RandomForestRegressor(max_depth=10)
model_forest.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_forest = model_forest.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_forest = model_forest.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 16423319.083621074
Testing Error: 15995566.473672068


In [8]:
model_forest_coefs = pd.DataFrame(
    model_forest.feature_importances_,
    index=list(trainX.columns), columns=['coef']
)
model_forest_coefs.sort_values(by=['coef'], key=abs, ascending=False)

,coef
weight_kg,4.081432e-01
sku_1663717,1.156731e-01
sku_1779728,5.732722e-02
group_4G130160,5.079679e-02
sku_762385,4.698893e-02
Poste de travail_40940,4.364495e-02
sku_762388,4.050617e-02
sku_1099285,2.733238e-02
sku_1099287,2.443214e-02
sku_1390940,2.281073e-02


# Weight

In [9]:
data_unit = factory_prod.drop(['Confirmation', 'Ordre', 'at'], axis=1)
data_unit_num = data_unit.select_dtypes(include=['int64', 'float64']).drop(['weight_kg'], axis=1)
data_unit_cat = data_unit.select_dtypes(exclude=['int64', 'float64'])
n_train = math.floor(0.9 * len(data_unit))

# split into train / test
trainX_num = data_unit_num[0:n_train]
trainY = data_unit[['weight_kg']][0:n_train]

testX_num = data_unit_num[n_train:]
testY = data_unit[['weight_kg']][n_train:]

# apply normalization and one-hot encoding
scaler = MinMaxScaler()
trainX_num = scaler.fit_transform(trainX_num)
testX_num = scaler.transform(testX_num)

data_unit_cat = pd.get_dummies(data_unit_cat)

# get final train and test set
data_scaled = pd.DataFrame(
    np.concatenate((np.concatenate((trainX_num, testX_num), axis=0), data_unit_cat), axis=1),
    columns=list(data_unit_num.columns) + list(data_unit_cat.columns)
)

trainX = data_scaled[0:n_train]
testX = data_scaled[n_train:]

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(8443, 534) (8443, 1)
(939, 534) (939, 1)


In [10]:
# fit decision tree model
model_tree = DecisionTreeRegressor(max_depth=40)
model_tree.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_tree = model_tree.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_tree = model_tree.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 83.0895632590248
Testing Error: 172.6312921178736


In [11]:
# fit random forest model
model_forest = RandomForestRegressor(max_depth=40)
model_forest.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_forest = model_forest.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_forest = model_forest.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 83.0895632590248
Testing Error: 172.6312921178736


In [12]:
model_weight_coefs = pd.DataFrame(
    np.concatenate((model_tree.feature_importances_.reshape(-1,1), model_forest.feature_importances_.reshape(-1,1)), axis=1),
    index=list(trainX.columns), columns=['coef_tree', 'coef_forest']
)

model_weight_coefs.sort_values(by=['coef_forest'], key=abs, ascending=False)

,coef_tree,coef_forest
units,4.938589e-01,5.274253e-01
Poste de travail_40904,0.000000e+00,1.446074e-01
group_4G200100,3.126822e-01,1.278273e-01
sku_1633466,2.981702e-02,1.988506e-02
group_4G140400,2.014445e-02,1.642238e-02
group_4G140120,1.669345e-02,1.575676e-02
group_4G130410,1.279142e-02,1.265414e-02
group_4G130310,1.208237e-02,1.112442e-02
sku_1589487,1.073258e-02,7.750892e-03
Poste de travail_40443,4.243333e-04,6.021577e-03


# Scrap

In [13]:
data_unit = factory_prod.drop(['Confirmation', 'Ordre', 'at'], axis=1)
data_unit_num = data_unit.select_dtypes(include=['int64', 'float64']).drop(['scrap'], axis=1)
data_unit_cat = data_unit.select_dtypes(exclude=['int64', 'float64'])
n_train = math.floor(0.9 * len(data_unit))

# split into train / test
trainX_num = data_unit_num[0:n_train]
trainY = data_unit[['scrap']][0:n_train]

testX_num = data_unit_num[n_train:]
testY = data_unit[['scrap']][n_train:]

# apply normalization and one-hot encoding
scaler = MinMaxScaler()
trainX_num = scaler.fit_transform(trainX_num)
testX_num = scaler.transform(testX_num)

data_unit_cat = pd.get_dummies(data_unit_cat)

# get final train and test set
data_scaled = pd.DataFrame(
    np.concatenate((np.concatenate((trainX_num, testX_num), axis=0), data_unit_cat), axis=1),
    columns=list(data_unit_num.columns) + list(data_unit_cat.columns)
)

trainX = data_scaled[0:n_train]
testX = data_scaled[n_train:]

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(8443, 534) (8443, 1)
(939, 534) (939, 1)


In [14]:
# fit decision tree model
model_tree = DecisionTreeRegressor(max_depth=10)
model_tree.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_tree = model_tree.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_tree = model_tree.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 1482.6301111050077
Testing Error: 6010.224338763769


In [15]:
# fit random forest model
model_forest = RandomForestRegressor(max_depth=10)
model_forest.fit(trainX, trainY)

# evaluate model with RMSE
train_preds_forest = model_forest.predict(trainX).reshape(-1,1)
print('Training Error:', math.sqrt( metrics.mean_squared_error( trainY, train_preds_tree ) ) )

test_preds_forest = model_forest.predict(testX).reshape(-1,1)
print('Testing Error:', math.sqrt( metrics.mean_squared_error( testY , test_preds_tree ) ) )

Training Error: 1482.6301111050077
Testing Error: 6010.224338763769


In [16]:
model_scrap_coefs = pd.DataFrame(
    np.concatenate((model_tree.feature_importances_.reshape(-1,1), model_forest.feature_importances_.reshape(-1,1)), axis=1),
    index=list(trainX.columns), columns=['coef_tree', 'coef_forest']
)

model_scrap_coefs.sort_values(by=['coef_forest'], key=abs, ascending=False)

,coef_tree,coef_forest
units,0.587793,4.157451e-01
weight_kg,0.297996,3.298863e-01
Centre de coûts_208006,0.010346,1.111379e-02
Poste de travail_40135,0.011138,1.092105e-02
sku_1397615,0.002364,9.984234e-03
sku_891546,0.010109,9.597723e-03
group_4G120410,0.000000,9.432141e-03
sku_776288,0.004134,9.322287e-03
Poste de travail_40438,0.008688,8.831412e-03
group_4G120110,0.000000,8.733281e-03
